# Deep Neural Network Using Back Propagation
---

In [1]:
import time
import numpy as np

## Utility Functions

In [2]:
def _t(x):
    return np.transpose(x)

def _m(A, B):
    return np.matmul(A, B)

## Sigmoid

$f(x)=1/(1+e^{-x})$
\
$\frac{df(x)}{dx}=f(x)(1-f(x))$

In [3]:
class Sigmoid: # To apply dynamic programming, define Sigmoid
               # in terms of 'class' not 'function'.
               # For dynamic programming, the memory in which
               # values for Back Propagation are stored is needed.
    def __init__(self):
        # Set the initial value of self.last_output as 1.
        self.last_output = 1
        
    def __call__(self, x):
        # self.last_output is calculated and stored during
        # forward inference.
        self.last_output = 1 / (1.0 + np.exp(-x))
        return self.last_output
        
    def grad(self):
        # The grad value is calculated and stored during
        # forward inference.
        return self.last_output * (1 - self.last_output)

## Mean Squared Errors

In [4]:
class MeanSquaredErrors:
    def __init__(self):
        self.dh = 1
        self.last_diff = 1
        
    def __call__(self, h, y):
        self.last_diff = h - y
        return 1 / 2 * np.mean(np.square(h - y))
        
    def grad(self):
        # self.last_diff (the grad value) is calculated and stored
        # during forward inference.
        return self.last_diff


## Neuron

In [5]:
class Neurons: # as a matter of a layer
    def __init__(self, W, b, a_obj):
        self.W = W
        self.b = b
        self.a = a_obj() # a: activation function
        
        # instances for gradients
        self.dW = np.zeros_like(self.W) # for W (weights)
        self.db = np.zeros_like(self.b) # for b (biases)
        # 'self.dh' plays a key role as a chain in backprop.
        self.dh = np.zeros_like(self.W[0, :]) # for inputs
        
        # self.last_x: inputs for the current layer
        # That is, outputs from the previous layer..
        self.last_x = np.zeros((self.W.shape[0]))
        # self.last_h: outputs from the current layer
        self.last_h = np.zeros((self.W.shape[1]))
        
    # Forward Inference
    def __call__(self, x):
        self.last_x = x
        # affine
        self.last_h = _m(_t(self.W), x) + self.b
        # activation
        return self.a(self.last_h)
        
    # dh_output / dh_input (matrix)
    def grad_chain(self):
        # self.a.grad() is broadcast over self.W's rows.
        # row: the nth weight of neurons
        # Thus, the outcome is a matrix.
        return self.W * self.a.grad()
    
    # grad_W: dW (previously accumulated * da * inputs)
    # dh: (previously) accumulated (from the previous layer)
    def grad_W(self, dh):
        grad = np.ones_like(self.W)
        grad_a = self.a.grad() # da (the grad of the activation function)
        # Calculate dW for a single neuron in a layer.
        # grad.shape[1]: the number of its outputs (the number of neurons in a layer)
        for j in range(grad.shape[1]):
            # grad[:, j]: the jth dW in the current layer
            # dh[j]: the jth accumulated grad in the next layer (scalar)
            # (the jth row of dh)
            # grad_a[j]: the jth activation function's grad
            # in the current layer (scalar)
            # self.last_x: inputs from the previous layer
            grad[:, j] = dh[j] * grad_a[j] * self.last_x
        return grad
        
    # dy/db = accumulated * da/db
    def grad_b(self, dh):
        return dh * self.a.grad()

## Deep Neural Network Structure

In [6]:
class DNN:
    def __init__(self, hidden_depth, num_neurons, num_inputs, num_outputs, activation = Sigmoid):
        
        """
        num_neurons: the number of neurons in a layer
        """
        
        def init_var(i, o):
            # i: the number of rows (the number of inputs)
            # o: the number of columns (the number of neurons in a layer)
            return np.random.normal(0.0, 0.01, (i, o)), np.zeros((o,))
        
        self.sequence = list()
        # First Hidden Layer
        W, b = init_var(num_inputs, num_neurons)
        self.sequence.append(Neurons(W, b, activation))
        
        # Hidden Layer
        for index in range(hidden_depth):
            W, b = init_var(num_neurons, num_neurons)
            self.sequence.append(Neurons(W, b, activation))
            
        # Output Layer
        W, b = init_var(num_neurons, num_outputs)
        self.sequence.append(Neurons(W, b, activation))
        
    def __call__(self, x):
        for layer in self.sequence:
            x = layer(x)
        return x

    def calc_gradient(self, loss_obj):
        loss_obj.dh = loss_obj.grad()
        self.sequence.append(loss_obj)
        
        # Back Propagation
        for i in range(len(self.sequence) - 1, 0, -1):
            # i: (len(self.sequence) - 1) ~ 1
            l1 = self.sequence[i]
            l0 = self.sequence[i - 1]
            
            # This is where the chain rule between layers occurs.
            l0.dh = _m(l0.grad_chain(), l1.dh) 
            # For dW, the next layer's dh is required.
            l0.dW = l0.grad_W(l1.dh)
            # For db, the next layer's dh is required.
            l0.db = l0.grad_b(l1.dh)
        
        # to use call just above
        self.sequence.remove(loss_obj)

## Gradient Descent Training

In [7]:
def gradient_descent(network, x, y, loss_obj, alpha = 0.01):
    loss = loss_obj(network(x), y) # Forward inference
    network.calc_gradient(loss_obj) # Back Propagation
    for layer in network.sequence:
        layer.W += -alpha * layer.dW
        layer.b += -alpha * layer.db
    return loss

## Training

In [8]:
x = np.random.normal(0.0, 1.0, (10,))
y = np.random.normal(0.0, 1.0, (2,))

t = time.time()
dnn = DNN(hidden_depth = 5, num_neurons = 128,
         num_inputs = 10, num_outputs = 2, activation = Sigmoid)
loss_obj = MeanSquaredErrors()
for epoch in range(10000):
    loss = gradient_descent(dnn, x, y, loss_obj, alpha = 0.01)
    print(f'Epoch {epoch + 1}, Test Loss: {loss}.')
print(f'{time.time() - t} seconds elapsed.')

Epoch 1, Test Loss: 0.3638575112235839.
Epoch 2, Test Loss: 0.3489472451833566.
Epoch 3, Test Loss: 0.3345890178488015.
Epoch 4, Test Loss: 0.32088305922280974.
Epoch 5, Test Loss: 0.30790181890147666.
Epoch 6, Test Loss: 0.29569013696734553.
Epoch 7, Test Loss: 0.2842676099426608.
Epoch 8, Test Loss: 0.27363238780464233.
Epoch 9, Test Loss: 0.2637656310126376.
Epoch 10, Test Loss: 0.2546359944409392.
Epoch 11, Test Loss: 0.2462037061765258.
Epoch 12, Test Loss: 0.23842400722376345.
Epoch 13, Test Loss: 0.23124987572945016.
Epoch 14, Test Loss: 0.22463406477100897.
Epoch 15, Test Loss: 0.21853054089653284.
Epoch 16, Test Loss: 0.2128954334419111.
Epoch 17, Test Loss: 0.2076876049193943.
Epoch 18, Test Loss: 0.20286894087503884.
Epoch 19, Test Loss: 0.19840444076824448.
Epoch 20, Test Loss: 0.19426217400756493.
Epoch 21, Test Loss: 0.1904131495543271.
Epoch 22, Test Loss: 0.18683113439068644.
Epoch 23, Test Loss: 0.18349244577141255.
Epoch 24, Test Loss: 0.18037573427944267.
Epoch 25, T

Epoch 199, Test Loss: 0.11141011453139865.
Epoch 200, Test Loss: 0.11136840499112036.
Epoch 201, Test Loss: 0.11132714357605353.
Epoch 202, Test Loss: 0.11128632318118048.
Epoch 203, Test Loss: 0.1112459368483119.
Epoch 204, Test Loss: 0.11120597776237622.
Epoch 205, Test Loss: 0.11116643924781941.
Epoch 206, Test Loss: 0.1111273147651116.
Epoch 207, Test Loss: 0.11108859790735671.
Epoch 208, Test Loss: 0.11105028239700163.
Epoch 209, Test Loss: 0.11101236208264186.
Epoch 210, Test Loss: 0.11097483093591969.
Epoch 211, Test Loss: 0.11093768304851286.
Epoch 212, Test Loss: 0.11090091262920948.
Epoch 213, Test Loss: 0.11086451400106752.
Epoch 214, Test Loss: 0.110828481598655.
Epoch 215, Test Loss: 0.11079280996536922.
Epoch 216, Test Loss: 0.11075749375083165.
Epoch 217, Test Loss: 0.1107225277083563.
Epoch 218, Test Loss: 0.1106879066924896.
Epoch 219, Test Loss: 0.11065362565661868.
Epoch 220, Test Loss: 0.11061967965064634.
Epoch 221, Test Loss: 0.11058606381873085.
Epoch 222, Test L

Epoch 400, Test Loss: 0.10738559325679388.
Epoch 401, Test Loss: 0.10737604973285442.
Epoch 402, Test Loss: 0.10736655536905841.
Epoch 403, Test Loss: 0.10735710978480538.
Epoch 404, Test Loss: 0.10734771260343642.
Epoch 405, Test Loss: 0.10733836345218296.
Epoch 406, Test Loss: 0.10732906196211683.
Epoch 407, Test Loss: 0.10731980776810074.
Epoch 408, Test Loss: 0.10731060050873964.
Epoch 409, Test Loss: 0.10730143982633288.
Epoch 410, Test Loss: 0.10729232536682685.
Epoch 411, Test Loss: 0.10728325677976876.
Epoch 412, Test Loss: 0.10727423371826077.
Epoch 413, Test Loss: 0.10726525583891487.
Epoch 414, Test Loss: 0.1072563228018087.
Epoch 415, Test Loss: 0.10724743427044159.
Epoch 416, Test Loss: 0.10723858991169179.
Epoch 417, Test Loss: 0.10722978939577377.
Epoch 418, Test Loss: 0.10722103239619672.
Epoch 419, Test Loss: 0.10721231858972326.
Epoch 420, Test Loss: 0.10720364765632899.
Epoch 421, Test Loss: 0.10719501927916239.
Epoch 422, Test Loss: 0.10718643314450577.
Epoch 423, T

Epoch 599, Test Loss: 0.10612845608179275.
Epoch 600, Test Loss: 0.10612429174418796.
Epoch 601, Test Loss: 0.10612014158155934.
Epoch 602, Test Loss: 0.10611600552186597.
Epoch 603, Test Loss: 0.10611188349355513.
Epoch 604, Test Loss: 0.10610777542555822.
Epoch 605, Test Loss: 0.10610368124728667.
Epoch 606, Test Loss: 0.10609960088862791.
Epoch 607, Test Loss: 0.10609553427994114.
Epoch 608, Test Loss: 0.1060914813520536.
Epoch 609, Test Loss: 0.10608744203625647.
Epoch 610, Test Loss: 0.10608341626430103.
Epoch 611, Test Loss: 0.1060794039683948.
Epoch 612, Test Loss: 0.10607540508119767.
Epoch 613, Test Loss: 0.10607141953581814.
Epoch 614, Test Loss: 0.10606744726580965.
Epoch 615, Test Loss: 0.10606348820516671.
Epoch 616, Test Loss: 0.10605954228832136.
Epoch 617, Test Loss: 0.10605560945013953.
Epoch 618, Test Loss: 0.10605168962591735.
Epoch 619, Test Loss: 0.10604778275137769.
Epoch 620, Test Loss: 0.10604388876266653.
Epoch 621, Test Loss: 0.10604000759634956.
Epoch 622, Te

Epoch 796, Test Loss: 0.10551371517913699.
Epoch 797, Test Loss: 0.10551138466767715.
Epoch 798, Test Loss: 0.10550906011645977.
Epoch 799, Test Loss: 0.10550674150276418.
Epoch 800, Test Loss: 0.10550442880398471.
Epoch 801, Test Loss: 0.10550212199763023.
Epoch 802, Test Loss: 0.10549982106132323.
Epoch 803, Test Loss: 0.10549752597279903.
Epoch 804, Test Loss: 0.10549523670990542.
Epoch 805, Test Loss: 0.10549295325060153.
Epoch 806, Test Loss: 0.10549067557295738.
Epoch 807, Test Loss: 0.10548840365515323.
Epoch 808, Test Loss: 0.1054861374754786.
Epoch 809, Test Loss: 0.10548387701233193.
Epoch 810, Test Loss: 0.1054816222442197.
Epoch 811, Test Loss: 0.10547937314975574.
Epoch 812, Test Loss: 0.10547712970766066.
Epoch 813, Test Loss: 0.10547489189676115.
Epoch 814, Test Loss: 0.10547265969598936.
Epoch 815, Test Loss: 0.10547043308438205.
Epoch 816, Test Loss: 0.10546821204108019.
Epoch 817, Test Loss: 0.10546599654532825.
Epoch 818, Test Loss: 0.10546378657647343.
Epoch 819, Te

...

Epoch 9038, Test Loss: 0.10386772446721222.
Epoch 9039, Test Loss: 0.10386770791560217.
Epoch 9040, Test Loss: 0.1038676913677212.
Epoch 9041, Test Loss: 0.10386767482356811.
Epoch 9042, Test Loss: 0.1038676582831416.
Epoch 9043, Test Loss: 0.1038676417464404.
Epoch 9044, Test Loss: 0.10386762521346332.
Epoch 9045, Test Loss: 0.1038676086842091.
Epoch 9046, Test Loss: 0.1038675921586765.
Epoch 9047, Test Loss: 0.1038675756368642.
Epoch 9048, Test Loss: 0.10386755911877103.
Epoch 9049, Test Loss: 0.1038675426043957.
Epoch 9050, Test Loss: 0.10386752609373696.
Epoch 9051, Test Loss: 0.10386750958679361.
Epoch 9052, Test Loss: 0.10386749308356436.
Epoch 9053, Test Loss: 0.10386747658404795.
Epoch 9054, Test Loss: 0.1038674600882432.
Epoch 9055, Test Loss: 0.10386744359614879.
Epoch 9056, Test Loss: 0.10386742710776355.
Epoch 9057, Test Loss: 0.10386741062308616.
Epoch 9058, Test Loss: 0.10386739414211545.
Epoch 9059, Test Loss: 0.10386737766485012.
Epoch 9060, Test Loss: 0.103867361191288

Epoch 9235, Test Loss: 0.10386453425599843.
Epoch 9236, Test Loss: 0.10386451841552945.
Epoch 9237, Test Loss: 0.1038645025785533.
Epoch 9238, Test Loss: 0.10386448674506882.
Epoch 9239, Test Loss: 0.10386447091507492.
Epoch 9240, Test Loss: 0.10386445508857035.
Epoch 9241, Test Loss: 0.10386443926555404.
Epoch 9242, Test Loss: 0.10386442344602485.
Epoch 9243, Test Loss: 0.10386440762998161.
Epoch 9244, Test Loss: 0.10386439181742317.
Epoch 9245, Test Loss: 0.10386437600834839.
Epoch 9246, Test Loss: 0.1038643602027561.
Epoch 9247, Test Loss: 0.10386434440064525.
Epoch 9248, Test Loss: 0.1038643286020146.
Epoch 9249, Test Loss: 0.10386431280686302.
Epoch 9250, Test Loss: 0.1038642970151894.
Epoch 9251, Test Loss: 0.10386428122699261.
Epoch 9252, Test Loss: 0.10386426544227148.
Epoch 9253, Test Loss: 0.10386424966102485.
Epoch 9254, Test Loss: 0.10386423388325164.
Epoch 9255, Test Loss: 0.10386421810895066.
Epoch 9256, Test Loss: 0.1038642023381208.
Epoch 9257, Test Loss: 0.103864186570

Epoch 9433, Test Loss: 0.10386146453175604.
Epoch 9434, Test Loss: 0.10386144936110361.
Epoch 9435, Test Loss: 0.10386143419372613.
Epoch 9436, Test Loss: 0.10386141902962265.
Epoch 9437, Test Loss: 0.10386140386879203.
Epoch 9438, Test Loss: 0.10386138871123328.
Epoch 9439, Test Loss: 0.1038613735569453.
Epoch 9440, Test Loss: 0.10386135840592706.
Epoch 9441, Test Loss: 0.10386134325817749.
Epoch 9442, Test Loss: 0.10386132811369556.
Epoch 9443, Test Loss: 0.1038613129724802.
Epoch 9444, Test Loss: 0.10386129783453041.
Epoch 9445, Test Loss: 0.10386128269984508.
Epoch 9446, Test Loss: 0.10386126756842316.
Epoch 9447, Test Loss: 0.10386125244026365.
Epoch 9448, Test Loss: 0.10386123731536545.
Epoch 9449, Test Loss: 0.10386122219372758.
Epoch 9450, Test Loss: 0.10386120707534893.
Epoch 9451, Test Loss: 0.10386119196022846.
Epoch 9452, Test Loss: 0.10386117684836516.
Epoch 9453, Test Loss: 0.10386116173975796.
Epoch 9454, Test Loss: 0.10386114663440581.
Epoch 9455, Test Loss: 0.103861131

Epoch 9629, Test Loss: 0.1038585523933094.
Epoch 9630, Test Loss: 0.10385853784496364.
Epoch 9631, Test Loss: 0.10385852329969468.
Epoch 9632, Test Loss: 0.10385850875750151.
Epoch 9633, Test Loss: 0.10385849421838324.
Epoch 9634, Test Loss: 0.10385847968233886.
Epoch 9635, Test Loss: 0.10385846514936735.
Epoch 9636, Test Loss: 0.10385845061946787.
Epoch 9637, Test Loss: 0.10385843609263933.
Epoch 9638, Test Loss: 0.10385842156888081.
Epoch 9639, Test Loss: 0.10385840704819133.
Epoch 9640, Test Loss: 0.10385839253056998.
Epoch 9641, Test Loss: 0.10385837801601572.
Epoch 9642, Test Loss: 0.10385836350452762.
Epoch 9643, Test Loss: 0.10385834899610469.
Epoch 9644, Test Loss: 0.103858334490746.
Epoch 9645, Test Loss: 0.10385831998845059.
Epoch 9646, Test Loss: 0.10385830548921743.
Epoch 9647, Test Loss: 0.10385829099304562.
Epoch 9648, Test Loss: 0.10385827649993419.
Epoch 9649, Test Loss: 0.10385826200988214.
Epoch 9650, Test Loss: 0.10385824752288855.
Epoch 9651, Test Loss: 0.1038582330

Epoch 9831, Test Loss: 0.10385567480384963.
Epoch 9832, Test Loss: 0.10385566085786552.
Epoch 9833, Test Loss: 0.10385564691477027.
Epoch 9834, Test Loss: 0.10385563297456299.
Epoch 9835, Test Loss: 0.10385561903724284.
Epoch 9836, Test Loss: 0.10385560510280885.
Epoch 9837, Test Loss: 0.10385559117126016.
Epoch 9838, Test Loss: 0.1038555772425959.
Epoch 9839, Test Loss: 0.10385556331681517.
Epoch 9840, Test Loss: 0.10385554939391707.
Epoch 9841, Test Loss: 0.10385553547390074.
Epoch 9842, Test Loss: 0.10385552155676522.
Epoch 9843, Test Loss: 0.10385550764250971.
Epoch 9844, Test Loss: 0.10385549373113329.
Epoch 9845, Test Loss: 0.103855479822635.
Epoch 9846, Test Loss: 0.10385546591701406.
Epoch 9847, Test Loss: 0.10385545201426956.
Epoch 9848, Test Loss: 0.10385543811440055.
Epoch 9849, Test Loss: 0.10385542421740622.
Epoch 9850, Test Loss: 0.1038554103232856.
Epoch 9851, Test Loss: 0.1038553964320379.
Epoch 9852, Test Loss: 0.10385538254366217.
Epoch 9853, Test Loss: 0.103855368658